In [2]:
from pyvi import ViTokenizer, ViPosTagger # thư viện NLP tiếng Việt
from tqdm import tqdm
import numpy as np
import gensim # thư viện NLP

In [3]:
def get_data(file):
    x = []
    y = []
    with open(file, 'r', encoding='utf8') as f:
        for line in f:
            i=14
            while(line[i]!=' '):
                i=i+1
            # lấy nhãn và nội dung
            label=line[14:i]
            content=line[i+1:]
            # xóa ký tự đặc biệt và tách từ
            content = gensim.utils.simple_preprocess(content)
            content = ' '.join(content)
            content = ViTokenizer.tokenize(content)
            x.append(content)
            y.append(label)
            
    return x,y
train_file = 'sell_detection_train.v1.0.txt'
x_data, y_data = get_data(train_file)
print(y_data)

['khong_ban', 'khong_ban', 'khong_ban', 'ban_hang', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'ban_hang', 'ban_hang', 'ban_hang', 'khong_ban', 'ban_hang', 'ban_hang', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'ban_hang', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'ban_hang', 'ban_hang', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'ban_hang', 'khong_ban', 'khong_ban', 'ban_hang', 'khong_ban', 'ban_hang', 'khong_ban', 'khong_ban', 'ban_hang', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'ban_hang', 'khong_ban', 'khong_ban', 'ban_hang', 'khong_ban', 'khong_ban', 'khong_ban', 'ban_hang', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'khong_ban', 'ban_hang', 'khong_ban', 'khong_ban', 'ban_hang', 'khong_ban', 'khong_ban', 'khong_ban', 'kh

In [5]:
import pickle
# lưu dữ liệu
pickle.dump(x_data, open('data/x_train.pkl', 'wb'))
pickle.dump(y_data, open('data/y_train.pkl', 'wb'))

In [7]:
test_file = 'sell_detection_test.v1.0.txt'
x_test = []
with open(test_file, 'r', encoding='utf8') as f:
    for line in f:
        line = gensim.utils.simple_preprocess(line)
        line = ' '.join(line)
        line = ViTokenizer.tokenize(line)
        x_test.append(line)
print(x_test[1])
pickle.dump(x_test, open('data/x_test.pkl', 'wb'))

minh la tai nguyen


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
import pickle

x_train = pickle.load(open('data/x_train.pkl', 'rb'))
y_train = pickle.load(open('data/y_train.pkl', 'rb'))

x_test = pickle.load(open('data/x_test.pkl', 'rb'))

# trích xuất đặc trưng
my_stop_words = ['thi', 'la', 'dc', 'bi', 'gi', 'thì', 'là', 'được', 'đc', 'bị', 'gì', 'a', 'à', 'á']
# phương pháp tf-idf ngram level - số lượng từ đại diện tối đa 10000
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', stop_words=my_stop_words, max_features=10000, ngram_range=(2, 3))
tfidf_vect_ngram.fit(x_train)

x_train_tfidf_ngram =  tfidf_vect_ngram.transform(x_train)
x_test_tfidf_ngram =  tfidf_vect_ngram.transform(x_test)

# giảm chiều của dữ liệu
svd_ngram = TruncatedSVD(n_components=300, random_state=42)
svd_ngram.fit(x_train_tfidf_ngram)

x_train_tfidf_ngram_svd = svd_ngram.transform(x_train_tfidf_ngram)
x_test_tfidf_ngram_svd = svd_ngram.transform(x_test_tfidf_ngram)

In [30]:
from sklearn.metrics import accuracy_score, classification_report
def train_model(classifier, x_train, y_train, x_test, file_solution):
    
    classifier.fit(x_train, y_train)
    
    train_pred = classifier.predict(x_train)
        #val_predictions = classifier.predict(x_val)
    test_pred = classifier.predict(x_test)
    with open(file_solution, 'w') as f:
        for label in test_pred:
            f.write(label+'\n')

In [33]:
from sklearn.svm import SVC
train_model(SVC(gamma='scale'), x_train_tfidf_ngram_svd, y_data, x_test_tfidf_ngram_svd, 'sell_detection_team20_solution3.result.txt')

# Test accuracy:  0.9561739710048325
#              precision    recall  f1-score   support
#
#   ban_hang       0.89      0.75      0.81       766
#  khong_ban       0.96      0.99      0.98      5235

In [31]:
from sklearn.linear_model import LogisticRegression
train_model(LogisticRegression(solver='lbfgs'), x_train_tfidf_ngram, y_train, x_test_tfidf_ngram, 'sell_detection_team20_solution1.result.txt')
# Test accuracy:  0.952
#              precision    recall  f1-score   support
#
#    ban_hang       0.96      0.66      0.79      1324
#   khong_ban       0.95      1.00      0.97      8676

In [32]:
from sklearn.naive_bayes import MultinomialNB
train_model(MultinomialNB(), x_train_tfidf_ngram, y_train, x_test_tfidf_ngram, 'sell_detection_team20_solution2.result.txt')
# Test accuracy:  0.9618
#              precision    recall  f1-score   support
#
#    ban_hang       0.95      0.75      0.84      1324
#   khong_ban       0.96      0.99      0.98      8676